# Fase 4 - Validação Final (O Conselho dos Agentes)

Esta é a etapa final e mais crítica do pipeline de extração do SINKT. Aqui, submetemos o grafo densificado a um rigoroso processo de auditoria e saneamento.

**Objetivo:** Transformar o grafo "bruto" em um **Artefato de Conhecimento.

## Arquitetura: A Mesa Redonda Virtual
Em vez de múltiplos agentes desconexos, simulamos uma **Mesa Redonda** via Prompt Engineering com o **GPT-4o**. Cada aresta é debatida por 8 personas especializadas:

1.  **Pedagogical Proponent (O Professor):** Defende valor didático.
2.  **Technical Proponent (O Engenheiro):** Defende precisão técnica.
3.  **Redundancy Critic (O Otimizador):** Caça duplicatas.
4.  **Hallucination Hunter (O Cético):** Verifica veracidade.
5.  **Structural Architect (O Topólogo):** Verifica hierarquias.
6.  **The Ontologist (O Terminologista):** Padroniza termos.
7.  **The Refactorer (O Reparador):** Propõe correções em vez de apenas deletar.
8.  **The Judge (O Decisor):** Bate o martelo (KEEP, DISCARD, REFACTOR).

## Pipeline de Processamento
1.  **Carga:** Importar `enhanced_graph.json`.
2.  **Debate:** Processamento em lote das arestas via LLM.
3.  **Refatoração:** Aplicar as correções sugeridas pelo 'Refactorer'.
4.  **Cirurgia Topológica:**
    *   Remoção de Ciclos (DAG Enforcement).
    *   Remoção de Nós Órfãos/Ilhas.
5.  **Entrega:** Geração do `final_sinkt_graph.json` e relatório de métricas.

In [1]:
!pip install -q langchain langchain-openai networkx pydantic tqdm python-dotenv

import os
import json
import networkx as nx
from typing import List, Literal, Optional
from tqdm import tqdm
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

load_dotenv()

# Configuração do Modelo
llm_judge = ChatOpenAI(model="gpt-5.1", temperature=0)

# --- NOVO DIRETÓRIO DE ENTRADA/SAÍDA ---
INPUT_FOLDER = "output/02_densification"
OUTPUT_FOLDER = "output/03_final_audit"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

INPUT_FILE = f"{INPUT_FOLDER}/enhanced_graph.json"
FINAL_FILE = f"{OUTPUT_FOLDER}/final_sinkt_graph.json"
DEBATE_LOG = f"{OUTPUT_FOLDER}/debate_log.md"

In [2]:
## 1. Carregamento dos Dados

def load_enhanced_graph():
    if not os.path.exists(INPUT_FILE):
        raise FileNotFoundError(f"Arquivo {INPUT_FILE} não encontrado. Execute o notebook 3 primeiro.")
    
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data['concepts'], data['relations']

concepts, relations = load_enhanced_graph()

print(f"Dados Carregados:")
print(f"   Conceitos: {len(concepts)}")
print(f"   Relações para Auditar: {len(relations)}")

Dados Carregados:
   Conceitos: 204
   Relações para Auditar: 219


In [3]:
## 2. O Motor de Debate (Prompt Engineering)

class EdgeVerdict(BaseModel):
    source: str
    target: str
    verdict: Literal['KEEP', 'DISCARD', 'REFACTOR']
    new_type: Optional[Literal['PREREQUISITE', 'PART_OF', 'IS_A', 'USE', 'RELATED_TO']] = Field(
        description="Obrigatório se verdict='REFACTOR'. O tipo de relação corrigido e padronizado."
    )
    direction_correction: Optional[Literal['FORWARD', 'REVERSE']] = Field(
        default='FORWARD', 
        description="Se 'REVERSE', indica que a seta deve ser invertida (Target -> Source)."
    )
    confidence: float = Field(description="Nível de confiança no veredito (0.0 a 1.0).")
    reason: str = Field(description="Resumo executivo do debate. Ex: 'O Topólogo vetou pois criava ciclo, o Engenheiro sugeriu inversão'.")

class DebateOutput(BaseModel):
    verdicts: List[EdgeVerdict]

# --- CORREÇÃO: Inicializar o parser ---
parser = PydanticOutputParser(pydantic_object=DebateOutput)

debate_system_prompt = """
Você é o **SINKT ORACLE**, o autoridade suprema na validação de Grafos de Conhecimento Educacional para Linux.
Sua missão é sanear o grafo final, garantindo integridade topológica e pedagógica.

Para CADA relação recebida, você deve orquestrar um **Debate Virtual Rápido** entre 8 especialistas. Não pule etapas.

### AS 8 PERSONAS DA MESA:

1.  **Professor:** Foca na *Causalidade Pedagógica*. "Aprender A desbloqueia B? Se não, descarte PREREQUISITE."
2.  **Engenheiro:** Foca na *Verdade Técnica*. "O comando 'ls' realmente faz parte do 'kernel'? Não! Descarte."
3.  **Otimizador:** Caça *Redundâncias*. "Se A é parte de B, e B é parte de C, precisamos de A->C? Talvez não."
4.  **Cético:** Caça *Alucinações*. "O conceito 'Linux' é um 'Comando'? Não. REFACTOR para 'Entidade'."
5.  **Topólogo (CRÍTICO):** Protege o *DAG*. "Essa relação cria um ciclo? A hierarquia flui do Geral para o Específico? Se A depende de B, B não pode depender de A."
6.  **Terminologista:** Padroniza. "Use 'PREREQUISITE' apenas para bloqueios de aprendizado. Use 'USE' para ferramentas."
7.  **Reparador:** Tenta salvar a aresta. "A relação está certa mas a direção errada? Inverta! O tipo está fraco? Fortaleça!"
8.  **JUIZ:** Sintetiza o debate e emite o veredito final com base na maioria qualificada e segurança técnica.

### REGRAS DE DECISÃO (Guidelines):

* **KEEP:** A relação é tecnicamente verdadeira, pedagogicamente útil e topologicamente segura.
* **REFACTOR:**
    * **Erro de Direção:** Ex: "Shell" PART_OF "Bash" (Errado) -> Inverter para "Bash" IS_A "Shell".
    * **Erro de Tipo:** Ex: "ls" PREREQUISITE "Terminal" (Fraco) -> Mudar para "Terminal" USE "ls" ou vice-versa.
* **DISCARD:**
    * Alucinações (fatos falsos).
    * Conexões muito genéricas que poluem o grafo (Ex: "Linux" RELATED_TO "Computador").
    * Ciclos óbvios em pré-requisitos.

### TIPOS CANÔNICOS PERMITIDOS:
1.  **PREREQUISITE**: Dependência forte de aprendizado. (Nó A deve ser aprendido antes de B).
2.  **PART_OF**: Composição mereológica. (Nó A é um componente do Nó B).
3.  **IS_A**: Taxonomia/Herança. (Nó A é um tipo específico do Nó B).
4.  **USE**: Relação funcional. (Ferramenta A utiliza/manipula Recurso B).
5.  **RELATED_TO**: Último caso. Use apenas se houver relação forte mas que não cabe nas acima.

Retorne o JSON estrito com a decisão final.
"""
debate_chain = (
    ChatPromptTemplate.from_messages([
        ("system", debate_system_prompt),
        ("user", """Aqui está o lote de arestas para julgamento.
        
        Lembre-se: O SINKT depende de um grafo limpo. Na dúvida, seja conservador.

        ARESTAS:
        {edges}

        {format_instructions}""")
    ])
    | llm_judge 
    | parser
)

In [4]:
## 3. Execução do Debate em Lote

BATCH_SIZE = 15 # Tamanho do lote para o GPT-4o
validated_edges = []
discarded_count = 0
refactored_count = 0

print(f"Iniciando Sessão do Conselho Jedi (GPT-4o)...")

# Preparar dados para o prompt
edge_strings = [f"{r['source']} -> {r.get('type', 'UNKNOWN')} -> {r['target']}" for r in relations]

for i in tqdm(range(0, len(edge_strings), BATCH_SIZE)):
    batch = edge_strings[i:i+BATCH_SIZE]
    
    try:
        result = debate_chain.invoke({
            "edges": "\n".join(batch),
            "format_instructions": PydanticOutputParser(pydantic_object=DebateOutput).get_format_instructions()
        })
        
        for v in result.verdicts:
            if v.verdict == 'DISCARD':
                discarded_count += 1
            else:
                final_type = v.new_type if v.new_type else 'RELATED_TO'
                if v.verdict == 'REFACTOR':
                    refactored_count += 1
                
                validated_edges.append({
                    "source": v.source,
                    "target": v.target,
                    "type": final_type,
                    "reason": v.reason
                })
                
    except Exception as e:
        print(f"❌ Erro no batch {i}: {e}")
        # Fallback: Em caso de erro de API, mantemos as arestas originais como RELATED_TO por segurança
        # ou descartamos. Aqui, vamos logar e pular para não paralisar.
        pass

print("\n=== Resultado do Debate ===")
print(f"Aprovadas: {len(validated_edges)}")
print(f"Descartadas: {discarded_count}")
print(f"Refatoradas: {refactored_count}")

Iniciando Sessão do Conselho Jedi (GPT-4o)...


 60%|██████    | 9/15 [03:11<02:13, 22.33s/it]

❌ Erro no batch 120: Failed to parse DebateOutput from completion {"verdicts": [{"source": "help", "target": "ls", "verdict": "REFACTOR", "new_type": "USE", "direction_correction": "FORWARD", "confidence": 0.86, "reason": "Professor e Terminologista apontam que o conceito é o mecanismo de ajuda genérico (‘help’ como ação/função) utilizando o comando específico ‘ls’ como exemplo; Engenheiro valida que é uma relação de uso funcional, não pré‑requisito. Topólogo vê que não cria ciclo relevante. JUIZ: manter aresta como USE está ok, apenas tipificando como USE canonical."}, {"source": "help", "target": "--help", "verdict": "REFACTOR", "new_type": "RELATED_TO", "direction_correction": "FORWARD", "confidence": 0.78, "reason": "Professor lembra que ‘help’ (conceito genérico de ajuda) não depende pedagogicamente de ‘--help’, é apenas uma forma concreta de obtê‑la. Engenheiro: ‘--help’ é uma opção de muitos comandos, não algo que ‘help’ use ativamente. Terminologista: não é PREREQUISITE nem USE

100%|██████████| 15/15 [05:04<00:00, 20.32s/it]


=== Resultado do Debate ===
Aprovadas: 145
Descartadas: 59
Refatoradas: 85


In [5]:
## 4. Construção do Grafo Final

print("\nConstruindo grafo final com todas as arestas validadas...")

# Construir Grafo NetworkX
G = nx.DiGraph()
for c in concepts:
    G.add_node(c['nome'], tipo=c['tipo'], definicao=c['definicao'])

# Adicionar arestas validadas
# Mantemos todas as arestas aprovadas pelo Conselho, sem filtragem topológica (ciclos/ilhas permitidos)
for r in validated_edges:
    if G.has_node(r['source']) and G.has_node(r['target']):
        G.add_edge(r['source'], r['target'], type=r['type'], reason=r['reason'])

print(f"Grafo construído com sucesso.")
print(f"Nós: {G.number_of_nodes()}")
print(f"Arestas: {G.number_of_edges()}")


Construindo grafo final com todas as arestas validadas...
Grafo construído com sucesso.
Nós: 204
Arestas: 122


In [6]:
## 5. Consolidação e Relatório Final

# Preparar JSON Final
final_concepts = []
for n, attr in G.nodes(data=True):
    final_concepts.append({
        "nome": n,
        "tipo": attr.get('tipo', 'Concept'),
        "definicao": attr.get('definicao', '')
    })

final_relations = []
for u, v, attr in G.edges(data=True):
    final_relations.append({
        "source": u,
        "target": v,
        "type": attr.get('type', 'RELATED_TO'),
        "reason": attr.get('reason', 'Validated by Council')
    })

output_data = {
    "metadata": {
        "pipeline": "SINKT v2 (Extraction -> Densification -> Council Validation)",
        "model": "GPT-4o",
        "nodes": G.number_of_nodes(),
        "edges": G.number_of_edges(),
        "density": nx.density(G)
    },
    "concepts": final_concepts,
    "relations": final_relations
}

with open(FINAL_FILE, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=4, ensure_ascii=False)

print("="*40)
print("RELATÓRIO FINAL DO SINKT")
print("="*40)
print(f"Arquivo Salvo: {FINAL_FILE}")
print(f"Nós Finais: {G.number_of_nodes()}")
print(f"Arestas Finais: {G.number_of_edges()}")
print(f"Densidade Final: {nx.density(G):.5f}")
print(f"Ciclos Restantes: {len(list(nx.simple_cycles(G)))}")

RELATÓRIO FINAL DO SINKT
Arquivo Salvo: output/03_final_audit/final_sinkt_graph.json
Nós Finais: 204
Arestas Finais: 122
Densidade Final: 0.00295
Ciclos Restantes: 6
